In [3]:
from torcheeg.datasets import DEAPDataset
from torcheeg import transforms
from torcheeg.datasets.constants import DEAP_CHANNEL_LOCATION_DICT

In [4]:

dataset = DEAPDataset(
    io_path='./temp/deap',
    root_path='/Users/andrei/Downloads/deap-dataset/data_preprocessed_python',
    offline_transform=transforms.Compose([
        transforms.BandDifferentialEntropy(apply_to_baseline=True),
        transforms.ToGrid(DEAP_CHANNEL_LOCATION_DICT, apply_to_baseline=True)
    ]),
    online_transform=transforms.Compose(
        [transforms.BaselineRemoval(),
         transforms.ToTensor()]),
    label_transform=transforms.Compose([
        transforms.Select('valence'),
        transforms.Binary(5.0),
    ]),
    num_worker=8)

[2025-08-15 22:23:51] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from ./temp/deap.


In [5]:
from torcheeg.model_selection import KFoldGroupbyTrial

k_fold = KFoldGroupbyTrial(n_splits=5,
                           split_path='./temp/deap-split',
                           shuffle=True,
                           random_state=42)

In [6]:
from torcheeg.models import CCNN


class Extractor(CCNN):
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.flatten(start_dim=1)
        return x

In [8]:
from torch.utils.data import DataLoader
from torcheeg.trainers import ClassifierTrainer, CWGANGPTrainer
from torcheeg.models import CCNN, BCGenerator, BCDiscriminator

import pytorch_lightning as pl

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    classifier = CCNN(num_classes=2, in_channels=4, grid_size=(9, 9))

    # if you have a pre-trained classifier, you can just load it, instead of training it from scratch
    trainer = ClassifierTrainer(model=classifier,
                                num_classes=2,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator="gpu")
    trainer.fit(train_loader,
                val_loader,
                max_epochs=20,
                default_root_dir=f'./temp/deap-model/{i}',
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)

    extractor = Extractor(num_classes=2, in_channels=4, grid_size=(9, 9))
    extractor.load_state_dict(classifier.state_dict())

    g_model = BCGenerator(in_channels=128)
    d_model = BCDiscriminator(in_channels=4)

    trainer = CWGANGPTrainer(g_model,
                              d_model,
                              metric_extractor=extractor,
                              metric_classifier=classifier,
                              metric_num_features=9 * 9 * 64,
                              metrics=['is'],
                             accelerator='gpu')
    trainer.fit(train_loader, val_loader, max_epochs=1)
    trainer.test(val_loader)

[2025-08-15 22:51:09] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./temp/deap-split.
[2025-08-15 22:51:09] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | CCNN             | 6.2 M  | train
1 | ce_fn         | CrossEntropyLoss | 0      | train
2 | train_loss    | MeanMetric       | 0      | train
3 | val_loss      | MeanMetric       | 0      | train
4 | test_loss     | MeanMetric       | 0      | train
5 | train_metrics | MetricCollection | 0      | train
6 | val_metrics   | MetricCollection | 0      | train
7 | test_metrics  | MetricCollection | 0      | train
------------------------

Epoch 0:   0%|          | 1/960 [00:00<01:03, 15.01it/s]

/Users/andrei/miniforge3/envs/tf/lib/python3.11/site-packages/torch/nn/functional.py:1535: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 0: 100%|██████████| 960/960 [00:40<00:00, 23.89it/s, v_num=2, train_loss=0.318, train_accuracy=0.891]

[2025-08-15 22:51:50] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.481 train_accuracy: 0.790 


...

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/andrei/miniforge3/envs/tf/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.



Testing DataLoader 0: 100%|██████████| 240/240 [00:06<00:00, 34.99it/s]

[2025-08-16 00:07:40] INFO (torcheeg/MainThread) 
[TEST] test_g_loss: 158.950 test_d_loss: 1.684 test_is: 1.000 



Testing DataLoader 0: 100%|██████████| 240/240 [00:06<00:00, 34.83it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_d_loss        │     1.684166431427002     │
│        test_g_loss        │    158.95030212402344     │
│          test_is          │     1.000223159790039     │
└───────────────────────────┴───────────────────────────┘